In [ ]:
import os
import sys
spark_path = os.environ['SPARK_HOME']
sys.path.append(spark_path + "/bin")
sys.path.append(spark_path + "/python")
sys.path.append(spark_path + "/python/pyspark/")
sys.path.append(spark_path + "/python/lib")
sys.path.append(spark_path + "/python/lib/pyspark.zip")
sys.path.append(spark_path + "/python/lib/py4j-0.10.9-src.zip")

import findspark
findspark.init()
import pyspark

In [ ]:
number_cores = 6
memory_gb = 16
conf = (pyspark.SparkConf().setMaster('local[{}]'.format(number_cores)).set('spark.driver.memory', '{}g'.format(memory_gb)))
sc = pyspark.SparkContext(conf=conf)

In [ ]:
!dir /users/trush/CSC496/PageRank

In [ ]:
! cat /users/trush/CSC496/PageRank/data/small_graph.dat

In [ ]:
graph_data = sc.textFile("/users/trush/CSC496/PageRank/data/small_graph.dat")
graph_data.take(10)

In [ ]:
# Incoming and outgoing links in a page
links = graph_data.map(lambda line: (line.split(" ")[0], line.split(" ")[1])) \
        .groupByKey() \
        .mapValues(list)
links.take(10)

In [ ]:
N = links.count()
ranks = links.map(lambda line: (line[0], 1/N))
ranks.take(N)

In [ ]:
votes = ranks.join(links)
votes.take(N)

In [ ]:
def calculateVotes(t):
    res = []
    for item in t[1][1]:
        count = len(t[1][1])
        res.append((item, t[1][0]/count))
    return res
# Weight of votes from original source
calculateVotes(('y', (0.3333333333333333, ['y', 'a'])))

In [ ]:
votes = ranks.join(links) \
        .flatMap(calculateVotes)
votes.collect()

In [ ]:
ranks.collect()

In [ ]:
ranks = votes.reduceByKey(lambda x, y: x + y)
ranks.collect()

In [ ]:
%%time
for i in range(10):
    votes = ranks.join(links) \
        .flatMap(calculateVotes)
    ranks = votes.reduceByKey(lambda x, y: x + y)
    print(ranks.collect())

In [ ]:
%%time

N = links.count()
ranks = links.map(lambda line: (line[0], 1/N))
ranks.take(N)
sum = 1

while sum > 0.1:
    old_ranks = ranks
    votes = ranks.join(links) \
            .flatMap(calculateVotes)
    ranks = votes.reduceByKey(lambda x, y: x + y)
    errors = old_ranks.join(ranks).mapValues(lambda v: abs(v[0] - v[1]))
    sum = errors.values().sum()
    print(sum)
ranks.collect()